In [18]:
from utlis import Guitar

In [19]:
class CAGEDSystem:
    def __init__(self, guitar):
        self.guitar = guitar
        # CAGED系统中的基础位置
        self.root_positions = {
            'C': {'shape': 'C', 'base_fret': 0},
            'A': {'shape': 'A', 'base_fret': 0},
            'G': {'shape': 'G', 'base_fret': 0},
            'E': {'shape': 'E', 'base_fret': 0},
            'D': {'shape': 'D', 'base_fret': 0}
        }
        self.shape_transitions = {
            'C': 'A',
            'A': 'G',
            'G': 'E',
            'E': 'D',
            'D': 'C'
        }
    
    def transpose_fingering(self, fingering, fret_shift):
        new_fingering = {}
        for string, fret in fingering.items():
            if fret == 0:  # 处理空弦：将其变为按弦
                new_fingering[string] = fret_shift
            else:
                new_fingering[string] = fret + fret_shift
        return new_fingering
    
    def get_caged_positions(self, chord_root):
        positions = []
        current_shape = None
        
        # 找到起始形状
        for shape_root, data in self.root_positions.items():
            if chord_root == shape_root:
                current_shape = data['shape']
                positions.append({
                    'shape': current_shape,
                    'position': data['base_fret'],
                    'fingering': self.guitar.cagedshape[shape_root]
                })
                break
        
        if not current_shape:
            return positions
        
        # 生成其他位置
        next_shape = self.shape_transitions[current_shape]
        fret_position = max(self.guitar.cagedshape[current_shape].values())
        
        while fret_position <= self.guitar.fretnum - 3:
            base_fingering = self.guitar.cagedshape[next_shape]
            new_fingering = self.transpose_fingering(base_fingering, fret_position)
            
            positions.append({
                'shape': next_shape,
                'position': fret_position,
                'fingering': new_fingering
            })
            
            next_shape = self.shape_transitions[next_shape]
            fret_position = max(new_fingering.values())
        
        return positions
    
    def find_nearest_position(self, chord_root, target_fret):
        positions = self.get_caged_positions(chord_root)
        if not positions:
            return None
            
        nearest_pos = min(positions, key=lambda x: abs(x['position'] - target_fret))
        return nearest_pos
    
    def get_chord_notes(self, position):
        return self.guitar.get_chord_midi(position['fingering'])


In [20]:
def demonstrate_caged():
    guitar = Guitar()
    caged = CAGEDSystem(guitar)
    
    chord = "C"
    positions = caged.get_caged_positions(chord)
    
    print(f"{chord}chord:")
    for pos in positions:
        print(f"\nshape: {pos['shape']}, position: {pos['position']}")
        print(f"- fingering: {pos['fingering']}")
        print(f"- MIDI: {caged.get_chord_notes(pos)}")

In [21]:
def demonstrate_nearest():
    guitar = Guitar()
    caged = CAGEDSystem(guitar)
    
    chord = "C"
    target_fret = 7
    
    nearest = caged.find_nearest_position(chord, target_fret)
    print(f"在{target_fret}品附近的{chord}和弦位置:")
    print(f"shape: {nearest['shape']}, position: {nearest['position']}")
    print(f"fingering: {nearest['fingering']}")
    print(f"MIDI: {caged.get_chord_notes(nearest)}")

In [22]:
demonstrate_caged()

Cchord:

shape: C, position: 0
- fingering: {6: 0, 5: 3, 4: 2, 3: 0, 2: 1, 1: 0}
- MIDI: [40, 48, 52, 55, 60, 64]

shape: A, position: 3
- fingering: {6: 3, 5: 3, 4: 5, 3: 5, 2: 5, 1: 3}
- MIDI: [43, 48, 55, 60, 64, 67]

shape: G, position: 5
- fingering: {6: 8, 5: 7, 4: 5, 3: 5, 2: 5, 1: 8}
- MIDI: [48, 52, 55, 60, 64, 72]

shape: E, position: 8
- fingering: {6: 8, 5: 10, 4: 10, 3: 9, 2: 8, 1: 8}
- MIDI: [48, 55, 60, 64, 67, 72]

shape: D, position: 10
- fingering: {6: 10, 5: 10, 4: 10, 3: 12, 2: 13, 1: 12}
- MIDI: [50, 55, 60, 67, 72, 76]

shape: C, position: 13
- fingering: {6: 13, 5: 16, 4: 15, 3: 13, 2: 14, 1: 13}
- MIDI: [53, 61, 65, 68, 73, 77]

shape: A, position: 16
- fingering: {6: 16, 5: 16, 4: 18, 3: 18, 2: 18, 1: 16}
- MIDI: [56, 61, 68, 73, 77, 80]

shape: G, position: 18
- fingering: {6: 21, 5: 20, 4: 18, 3: 18, 2: 18, 1: 21}
- MIDI: [61, 65, 68, 73, 77, 85]

shape: E, position: 21
- fingering: {6: 21, 5: 23, 4: 23, 3: 22, 2: 21, 1: 21}
- MIDI: [61, 68, 73, 77, 80, 85]


In [23]:
demonstrate_nearest()

在7品附近的C和弦位置:
shape: E, position: 8
fingering: {6: 8, 5: 10, 4: 10, 3: 9, 2: 8, 1: 8}
MIDI: [48, 55, 60, 64, 67, 72]
